# PyTorch基础

作为深度学习的框架，PyTorch的数据基础便是张量`tensor`类，这一点和Tensorflow、MxNet是一致的。

In [1]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
print(f'Import PyTorch V{torch.__version__}')

Import PyTorch V1.12.1


## 创建张量

和其他科学计算库一样，PyTorch提供了很快快捷的创建张量函数，不同的函数生成数据的逻辑不同，但输入的参数一般包括形状、数据类型和存在的设备平台。

主要的创建函数包括：

- `empty`：创建过程不对数据进行初始化，申请的内存空间是什么就是什么；
- `zeros`：数据初始化为0；
- `ones`：数据初始化为1；
- `rand`：数据按照0到1之间的平均分布随机初始化；
- `randn`：数据按照正态分布随机初始化，正态分布的默认均值为0，标准差为1。

In [2]:
dev1 = torch.device('cpu')
dev2 = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for func_name, shape, dtype, dev in [
    ('empty', (2,), torch.double, dev1),
    ('zeros', (1, 3), torch.float32, dev1),
    ('ones', (2, 1, 3), torch.int, dev2),
    ('rand', (2, 2), torch.double, dev2),
    ('randn', (4, 4), torch.float64, dev1),
]:
    func = getattr(torch, func_name)
    t = func(size=shape, dtype=dtype, device=dev)
    print(f'{func_name}, {shape}, {dtype}, {dev}')
    print(t)
    print()

empty, (2,), torch.float64, cpu
tensor([7.5657e-307, 4.0054e-307], dtype=torch.float64)

zeros, (1, 3), torch.float32, cpu
tensor([[0., 0., 0.]])

ones, (2, 1, 3), torch.int32, cuda
tensor([[[1, 1, 1]],

        [[1, 1, 1]]], device='cuda:0', dtype=torch.int32)

rand, (2, 2), torch.float64, cuda
tensor([[0.7521, 0.7281],
        [0.4589, 0.4376]], device='cuda:0', dtype=torch.float64)

randn, (4, 4), torch.float64, cpu
tensor([[-1.0218,  0.3259,  0.6338, -0.9613],
        [ 0.3029, -0.2843,  0.5592,  0.8057],
        [-1.5108,  0.3070,  0.5089,  0.1312],
        [-0.7727, -0.0278,  0.2772, -1.6303]], dtype=torch.float64)



## 转换

PyTorch中的张量可以由数组直接转换而来，也支持与NumPy数组的相互转换。

**注意**：NumPy不支持CUDA平台。

In [3]:
arr = [[0.2, 0.7, 1.7], [-23.0, 81.3, -44e-1]]
t_arr = torch.tensor(arr, dtype=torch.float32, device=dev2)

arr, t_arr

([[0.2, 0.7, 1.7], [-23.0, 81.3, -4.4]],
 tensor([[  0.2000,   0.7000,   1.7000],
         [-23.0000,  81.3000,  -4.4000]], device='cuda:0'))

In [4]:
t_arr.to('cpu').numpy()

array([[  0.2,   0.7,   1.7],
       [-23. ,  81.3,  -4.4]], dtype=float32)

In [5]:
ndarr = np.linspace(0.0, 10.0, 7)
ndarr, torch.from_numpy(ndarr)

(array([ 0.        ,  1.66666667,  3.33333333,  5.        ,  6.66666667,
         8.33333333, 10.        ]),
 tensor([ 0.0000,  1.6667,  3.3333,  5.0000,  6.6667,  8.3333, 10.0000],
        dtype=torch.float64))

## 简单运算

主要包括三类：

- 运算符的重载，如`+`、`-`、`*`等；
- 运算函数，例如`add`、`dot`、`mv`、`mm`等；
- 自运算函数，例如`add_`，在运算函数后加下划线，运算的第一个变量和最终结果的保留都是自身。

In [6]:
t0, t1, v0, v1, m0, m1 = \
    torch.rand(2, 3), torch.randn(2, 3), \
    torch.rand(4), torch.randn(4), \
    torch.rand(4, 4), torch.randn(4, 4)

In [7]:
t0 + t1, t0 / t1 + t0 * t1

(tensor([[ 1.7125,  1.1805, -0.5837],
         [-0.4170, -0.0729, -0.2057]]),
 tensor([[ 1.6350,  0.3591, -0.3992],
         [-1.2408, -0.6148, -1.7774]]))

In [8]:
torch.add(t0, t1), torch.add(torch.divide(t0, t1), torch.multiply(t0, t1))

(tensor([[ 1.7125,  1.1805, -0.5837],
         [-0.4170, -0.0729, -0.2057]]),
 tensor([[ 1.6350,  0.3591, -0.3992],
         [-1.2408, -0.6148, -1.7774]]))

In [9]:
v0 * v1, torch.dot(v0, v1)

(tensor([-0.0009, -0.3453, -0.6080,  0.0890]), tensor(-0.8652))

In [10]:
m0 * v0, torch.mv(m0, v0)

(tensor([[0.0113, 0.0016, 0.5771, 0.0667],
         [0.0350, 0.2411, 0.3935, 0.0477],
         [0.0818, 0.1721, 0.5972, 0.0090],
         [0.0650, 0.0265, 0.1586, 0.0502]]),
 tensor([0.6567, 0.7173, 0.8601, 0.3003]))

In [11]:
m0 * m1, torch.mm(m0, m1)

(tensor([[ 1.0169e-03,  5.1313e-03,  9.7446e-01,  1.1505e-01],
         [ 2.5670e-01,  1.8685e-01,  6.6122e-02,  1.5874e-02],
         [ 1.2535e+00, -1.5788e-01,  7.0175e-01, -1.9961e-02],
         [ 2.1085e-01,  3.2724e-02, -2.9733e-01, -9.6362e-01]]),
 tensor([[ 1.6109,  0.0598,  0.1664, -1.5079],
         [ 1.6467,  0.5190,  0.4532, -0.9829],
         [ 1.8980,  0.7303,  1.5254, -0.2404],
         [ 0.5912,  0.7897,  0.3985, -0.8923]]))

In [12]:
t0.add_(t1)
t0

tensor([[ 1.7125,  1.1805, -0.5837],
        [-0.4170, -0.0729, -0.2057]])

## 变换形状

PyTorch继承了NumPy中变换数组形状的函数`reshape`，同时还提供了另一个变换形状的函数`view`。

In [13]:
t0.reshape(3, 2), t0.view(3, 2)

(tensor([[ 1.7125,  1.1805],
         [-0.5837, -0.4170],
         [-0.0729, -0.2057]]),
 tensor([[ 1.7125,  1.1805],
         [-0.5837, -0.4170],
         [-0.0729, -0.2057]]))

In [14]:
m0.reshape(-1, 8), m0.view(-1, 8)

(tensor([[0.1119, 0.0048, 0.9214, 0.5369, 0.3450, 0.7291, 0.6283, 0.3838],
         [0.8077, 0.5203, 0.9535, 0.0727, 0.6418, 0.0800, 0.2532, 0.4045]]),
 tensor([[0.1119, 0.0048, 0.9214, 0.5369, 0.3450, 0.7291, 0.6283, 0.3838],
         [0.8077, 0.5203, 0.9535, 0.0727, 0.6418, 0.0800, 0.2532, 0.4045]]))

In [15]:
m1.view(8, -1)

tensor([[ 0.0091,  1.0605],
        [ 1.0576,  0.2143],
        [ 0.7440,  0.2563],
        [ 0.1052,  0.0414],
        [ 1.5518, -0.3035],
        [ 0.7359, -0.2746],
        [ 0.3285,  0.4088],
        [-1.1743, -2.3824]])

`view`除了可以用来改变形状，还可以改变数据类型。

In [16]:
m1.view(torch.double)

tensor([[ 1.1596e-02,  2.5541e-08],
        [ 1.4308e-07,  4.5106e-14],
        [-4.0780e-07, -2.1288e-07],
        [ 4.1257e-06, -6.1177e+00]], dtype=torch.float64)